### Notes
We might need to remove the common words in the corpus beside the stop words as those are common cross all corpus

In [1]:
import pandas as pd
import nltk as nltk

utterances_df = pd.read_csv('./temp_data/utterances.csv')
utterances_df.set_index('id')
utterances_df['actionbody'] = utterances_df['actionbody'].astype('str')
uttr_voc_df = pd.read_csv('./temp_data/utterances_voc.csv')
uttr_voc_df = uttr_voc_df.dropna()

In [2]:
uttr_voc_df.groupby('type').count().sort_values('word',ascending=False).to_csv('./temp_data/ph.csv')

In [3]:
ph_s = uttr_voc_df['type'].drop_duplicates()
ph_s[ph_s.str.contains(r'ph_[a-z^\s]+', regex = True)]

0             ph_technical
209           ph_mobile_no
9132               ph_user
9152               ph_name
10059              ph_code
               ...        
24666          yes ph_user
24670    yesterday ph_path
24683         your ph_path
24839          zip ph_path
24844         zoom ph_path
Name: type, Length: 322, dtype: object

In [4]:
# tech_words_df = uttr_voc_df[uttr_voc_df['type'].isin(['ph_technical','ph_name'])]
tech_words_df = uttr_voc_df
tech_words_df = tech_words_df.set_index('word')

In [5]:
utterances_df.head(5)
utterances_df.columns

Index(['id', 'issueid', 'created', 'author', 'actionbody', 'utr_seq',
       'is_private', 'comment_seq'],
      dtype='object')

In [6]:
rows = utterances_df.iterrows()
utterances_df['changed'] = False
utterances_df['actionbody_old'] = utterances_df['actionbody']
for idx,r in rows:
    body = r['actionbody']
    body_words = nltk.tokenize.wordpunct_tokenize(body)
    updated_body = ""
    affected = False
    for word in body_words:
        w = word
        if word in tech_words_df.index:
            w = tech_words_df.loc[w,'type']
            affected = True
        updated_body += w + " "  
    if affected:
        utterances_df.loc[idx,'actionbody'] = updated_body
        utterances_df.loc[idx,'changed'] = True

print('done')

done


In [7]:
utterances_df[['id', 'changed', 'actionbody_old','actionbody','issueid', 'created', 'author', 'utr_seq',
       'is_private', 'comment_seq']].set_index('id').to_csv('./temp_data/utterances_cleaned.csv')
print('done')        

done
